Making sense of the protest news data that was annotated via Prodigy's active learning (`textcat.teach`).

In [1]:
import json

with open(project_root / "data" / "protest" / "protest_news_wip.jsonl") as f:
    items = [json.loads(line) for line in f]

In [2]:
len(items)

1129

In [3]:
items[1]

{'text': 'Tel Aviv\n\nAngesichts steigender Infektionszahlen will Israel als erstes Land Über-60-Jährigen eine dritte Impfdosis gegen das Coronavirus geben. Es gilt zudem wieder der sogenannte Grüne Pass.\n\nRegierungschef Naftali Bennett teilte am Donnerstagabend mit, dies gelte für Patienten, die vor mindestens fünf Monaten ihre zweite Impfdosis erhalten haben. Ein Expertenteam in Israel hatte zuvor eine solche Auffrischungsimpfung mit dem Biontech-Pfizer-Präparat empfohlen, obwohl es noch keine entsprechende FDA-Vorgabe gibt. Bennett sagte, der 60 Jahre alte Staatspräsident Izchak Herzog werde sich am Freitagmorgen als Erster mit der dritten Dosis impfen lassen.\n\nDie allgemeine Impfkampagne in den Krankenkassen solle binnen weniger Tage beginnen. "Ich rufe ältere Menschen, die schon die zweite Dosis erhalten haben, sich auch zum dritten Mal impfen zu lassen", so Bennett. "Dies schützt vor schwerer Erkrankung und Tod." Nach Angaben des Regierungschefs haben in Israel bereits 2000 M

In [4]:
for item in items[:10]:
    print(item["text"][:100])
    print(item["label"])
    print(item["answer"])
    print()

2\. Dezember 2020 um 05:45 Uhr

##  Schutz : Bannmeile für den Bundestag

„Jedes Landesparlament ist
relevant
reject

Tel Aviv

Angesichts steigender Infektionszahlen will Israel als erstes Land Über-60-Jährigen eine d
relevant
reject

Eine Erde reicht der Menschheit nicht mehr zum Leben: So wurde errechnet, dass wir bereits am 2. Aug
relevant
reject

Für mich war es immer die Sendung des Jahres!» - Mit diesen Worten hat sich Moderator Günther Jauch 
relevant
reject

Tausende Menschen protestieren in Caracas gegen die venezolanische Regierung.

Bild: Rafael Hernande
relevant
reject

Diesen Tag wird Gerhard Deutschmann nicht so schnell vergessen. Schließlich wird das 61. Weihnachtsk
relevant
reject

Als zwei Polizisten im Mai 2020 wegen Maskenverweigerer in einen Troisdorfer Supermarkt gerufen werd
relevant
reject

##  Nun doch Suedlink-Trasse durch Thüringen

31.10.2020, 20:03 | Lesedauer: 2 Minuten

Protest gege
relevant
reject

Barcelona (dpa) - Die von einer Entmachtung bedrohte Reg

In [5]:
print(len([item for item in items if item["label"] == "relevant"]))
print(len([item for item in items if item["label"] == "irrelevant"]))
print(len([item for item in items if item["answer"] == "accept"]))
print(len([item for item in items if item["answer"] == "reject"]))

import numpy as np

confusion = np.zeros((2, 2), dtype=int)
for item in items:
    confusion[int(item["label"] != "relevant"), int(item["answer"] != "accept")] += 1
confusion  # y: pred relevant/irrelevant, x: accept/reject

846
283
358
771


array([[107, 739],
       [251,  32]])

In [6]:
def opposite_label(label):
    return "irrelevant" if label == "relevant" else "relevant"


for item in items:
    item["true_label"] = (
        item["label"] if item["answer"] == "accept" else opposite_label(item["label"])
    )

In [7]:
n_relevant = len([item for item in items if item["true_label"] == "relevant"])
n_irrelevant = len([item for item in items if item["true_label"] == "irrelevant"])
n_relevant, n_irrelevant

(139, 990)

In [8]:
ratio_relevant = n_relevant / (n_relevant + n_irrelevant)
ratio_relevant

0.12311780336581045

In [9]:
newspaper_counts = {}
for item in items:
    if item["true_label"] != "relevant":
        continue
    medium = item["meta"]["homepage"]
    newspaper_counts[medium] = newspaper_counts.get(medium, 0) + 240 / 5
sorted(newspaper_counts.items(), key=lambda x: x[1], reverse=True)

[('infranken.de', 1488.0),
 ('wn.de', 1008.0),
 ('zeit.de', 816.0),
 ('azonline.de', 624.0),
 ('op-online.de', 480.0),
 ('dzonline.de', 384.0),
 ('tz.de', 240.0),
 ('sauerlandkurier.de', 192.0),
 ('lz.de', 192.0),
 ('kreiszeitung.de', 144.0),
 ('muensterschezeitung.de', 144.0),
 ('bz-berlin.de', 144.0),
 ('nw.de', 96.0),
 ('nordbayern.de', 96.0),
 ('wz.de', 96.0),
 ('wa.de', 96.0),
 ('waz-online.de', 96.0),
 ('otz.de', 48.0),
 ('weser-kurier.de', 48.0),
 ('idowa.de', 48.0),
 ('nn.de', 48.0),
 ('aachener-nachrichten.de', 48.0),
 ('onetz.de', 48.0),
 ('augsburger-allgemeine.de', 48.0)]

In [10]:
import optuna

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///db.sqlite3",
    study_name="glpn-3",
    load_if_exists=True,
)
study.best_params

[I 2023-02-04 12:53:40,703] Using an existing study with name 'glpn-3' instead of creating a new one.


{'classifier': 'XGBClassifier',
 'feature_extraction': 'bow',
 'max_depth': 9,
 'max_features': 210,
 'max_ngram': 5,
 'n_estimators': 14}

In [15]:
from src.data.protests.detection.simple_classification import objective

model = objective(study.best_trial, return_model=True)

# predict
y_pred = model.predict([item["text"] for item in items])
y_true = [(1 if item["true_label"] == "relevant" else 0) for item in items]

# compute metrics
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9596    0.8394    0.8955       990
           1     0.3954    0.7482    0.5174       139

    accuracy                         0.8282      1129
   macro avg     0.6775    0.7938    0.7064      1129
weighted avg     0.8901    0.8282    0.8489      1129



In [16]:
import pandas as pd

df = pd.DataFrame({"text": [item["text"] for item in items], "label": y_true})
df.head()

,text,label
0,2\. Dezember 2020 um 05:45 Uhr\n\n## Schutz :...,0
1,Tel Aviv\n\nAngesichts steigender Infektionsza...,0
2,Eine Erde reicht der Menschheit nicht mehr zum...,0
3,Für mich war es immer die Sendung des Jahres!»...,0
4,Tausende Menschen protestieren in Caracas gege...,0


In [18]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.5, random_state=20230204)

# train on my data
model.fit(train["text"], train["label"])

# predict
y_pred = model.predict(test["text"])
y_true = test["label"]

# compute metrics
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9337    0.9801    0.9564       503
           1     0.7297    0.4355    0.5455        62

    accuracy                         0.9204       565
   macro avg     0.8317    0.7078    0.7509       565
weighted avg     0.9113    0.9204    0.9113       565

